### Brainstorming for potential data visualizations and some data preparation

In [1]:
#imports
import pandas as pd
import numpy as np

#graphing
# import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sns

# no warnings
import warnings
warnings.filterwarnings('ignore')

#other 
import json

### Data Sources

In [2]:
# import data from https://bakeoff.netlify.app/index.html)
baker_results = pd.read_csv('data/baker_results.csv',encoding = "ISO-8859-1")
# episode_results = pd.read_csv('data/episode_results.csv',encoding = "ISO-8859-1")
challenge_results = pd.read_csv('data/challenge_results.csv',encoding = "ISO-8859-1")
ratings_seasons = pd.read_csv('data/ratings_seasons.csv',encoding = "ISO-8859-1")

In [3]:
# import data from https://medium.com/analytics-vidhya/analyzing-the-great-british-bake-off-part-1-ffcdf3791bf3
gbbo_org = pd.read_csv('data/gbbo.csv')
gbbo = gbbo_org.iloc[:,:-2]
gbbo['Season'] = gbbo['Season'].str[7:].astype(int)

**average 7 day viewership by series**

In [4]:
# average 7 day viewership by series
viewers_by_season8 = ratings_seasons.groupby('series').mean().reset_index()[['series','viewers_7day']]

# add season 9 and 10 manually (https://en.wikipedia.org/wiki/The_Great_British_Bake_Off_(series_9))
avg_viewers9 = (9.55 + 9.31 + 8.91 + 8.88 + 8.67 + 8.91 + 9.22 + 9.69 + 9.5 + 10.34)/10
avg_viewers10 = (9.62 + 9.38 + 8.94 + 8.96 + 9.26 + 8.70 + 8.98 + 9.19 + 9.34 + 10.05)/10

# dataframe with series 9 and 10
avg_viewers910 = pd.DataFrame({'series':[9,10],'viewers_7day':[avg_viewers9,avg_viewers10]})

# put all series together
viewers_by_season = pd.concat([viewers_by_season8,avg_viewers910 ],axis=0)

In [5]:
# average 7 day viewership by series
# sns.pointplot(x='series',y='viewers_7day',data=viewers_by_season)

# export to json
dct = {'data': viewers_by_season.values.tolist()}
with open('../data/viewers_by_season.json', 'w') as file:
    json.dump(dct,file,indent=0)

**age breakdown of contestants**
- series 1-10

In [6]:
# helper function to group ages
def age_group(age):
    if age>=17 and age<30:
        return '17 to 30'
    if age>=30 and age<45:
        return '30 to 45'
    if age>=45 and age<60:
        return '45 to 59'
    else:
        return '60+'

In [7]:
# get only bakers
gbbo_bakers = gbbo.drop_duplicates(subset=['Season','Baker'])[['Season','Baker','Gender','Age']]

In [8]:
# plot pie plot of age breakdown
ages = gbbo_bakers['Age'].apply(age_group).value_counts(normalize=True)
# ages.plot(kind='pie',subplots=True)

In [9]:
# convert to dataframe
df = pd.DataFrame(ages).reset_index()
df.columns = ['name','y']

# export to json
data = list(df.apply(pd.Series.to_json,axis=1).apply(json.loads))
with open('../data/ages.json', 'w') as file:
    json.dump(data,file,indent=0)

**gender breakdown**

In [10]:
# plot pie plot of gender breakdown
gender = gbbo_bakers['Gender'].value_counts(normalize=True)
# gender.plot(kind='pie',subplots=True)

In [11]:
# convert to dataframe
df = pd.DataFrame(gender).reset_index()
df.columns = ['name','y']

#export to json
data = list(df.apply(pd.Series.to_json,axis=1).apply(json.loads))
with open('../data/gender.json', 'w') as file:
    json.dump(data,file,indent=0)

**word cloud of the bake titles**

In [12]:
# split bake titles
words = pd.Series(challenge_results['signature'].dropna().str.split().sum() + 
                      challenge_results['showstopper'].dropna().str.split().sum())
words = words.apply(lambda x:'Chocolate' if x == "Chocolate," else x)

# no common words
stopwords = ['and','&','with','a','of','The']
words = words[~ words.isin(pd.Series(stopwords))]

# first 150 words
words_150 = pd.DataFrame(words.value_counts()).reset_index()[:150]

# export json
words_150.columns = ['name','weight']
words_150 = words_150.apply(pd.Series.to_json,axis=1)
dct = {}
dct['words'] = list(words_150.apply(json.loads))
with open('../data/bake_words.json', 'w') as file:
    json.dump(dct,file,indent=0)

**Compare winners handshakes, technicals, star_baker**

In [13]:
# get winners for first data source
winners8 = baker_results[baker_results['series_winner']==1].sort_values('series')
winners8.head()

,series,baker_full,baker,age,occupation,hometown,baker_last,baker_first,star_baker,technical_winner,...,technical_median,series_winner,series_runner_up,total_episodes_appeared,first_date_appeared,last_date_appeared,first_date_us,last_date_us,percent_episodes_appeared,percent_technical_top3
2,1,"Edward ""Edd"" Kimber",Edd,24,Debt collector for Yorkshire Bank,Bradford,Kimber,Edward,0,2,...,2.0,1,0,6,2010-08-17,2010-09-21,NaN,NaN,100.0,66.666667
15,2,Joanne Wheatley,Joanne,41,Housewife,"Ongar, Essex",Wheatley,Joanne,1,3,...,4.5,1,0,8,2011-08-16,2011-10-04,NaN,NaN,100.0,50.000000
26,3,John Whaite,John,23,Law student,Wigan,Whaite,John,1,1,...,3.0,1,0,10,2012-08-14,2012-10-16,NaN,NaN,100.0,60.000000
38,4,Frances Quinn,Frances,31,Children's Clothes Designer,"Market Harborough, Leicestershire",Quinn,Frances,1,1,...,3.0,1,0,10,2013-08-20,2013-10-22,2014-12-28,2015-03-01,100.0,70.000000
56,5,Nancy Birtwhistle,Nancy,60,Retired practice manager,"Barton-Upon-Humber, Lincolnshire",Birtwhistle,Nancy,1,3,...,2.5,1,0,10,2014-08-06,2014-10-08,2015-09-06,2015-11-08,100.0,70.000000


In [14]:
# get s1-8 winners for second data source
w = gbbo[(gbbo['Winner']==1) | ((gbbo['Baker']=='David') & (gbbo['Season']==10))]
handshakes = w.groupby(
    'Baker').sum()[['Signature Handshake','Showstopper Handshake']]
winners_list = list(winners8['baker']) + ['Rahul','David']
handshakes.reindex(winners_list)

,Signature Handshake,Showstopper Handshake
Baker,,
Edd,0,0
Joanne,0,0
John,0,0
Frances,1,0
Nancy,0,0
Nadiya,1,0
Candice,1,0
Sophie,1,0
Rahul,2,1


In [15]:
rahul = gbbo[(gbbo['Baker']=='Rahul') & (gbbo['Season']==9)]
david = gbbo[(gbbo['Baker']=='David') & (gbbo['Season']==10)]

In [16]:
# get num of start bakers, technical wins, and handshakes
dct = {}
dct['bakers'] = winners_list
dct['star_baker'] = list(winners8['star_baker']) + [int(rahul['Star Baker'].sum()-1), # hard code to fix data inconsistencies
                                                    int(david['Star Baker'].sum()+1)]
dct['technical'] = list(winners8['technical_winner']) + [int((rahul['Technical Rank']==1).sum()),
                                                         int((david['Technical Rank']==1).sum())]
dct['handshake'] = list(handshakes['Signature Handshake'] + handshakes['Showstopper Handshake'])

# to json
with open('../data/winners_stats.json', 'w') as file:
    json.dump(dct,file,indent=0)

**bubble chart of winner based on instagram + twitter followers**

In [51]:
# data collected manually as of 6/6/20
insta = pd.DataFrame({'baker': ['Edd','Joanne','John','Frances','Nancy','Nadiya','Candice','Sophie','Rahul','David'],
                      'age': list(w.drop_duplicates('Baker')['Age']),
                      'occupation': list(winners8['occupation']) +['Engineering Researcher','Health Advisor'],
                      'hometown': list(winners8['hometown']) + ['Howrah, India','Whitby'],
                     'followers': [288+40.9, 26.3, 79.8+89.1, 49.1+74.6, 104+38.1, 520+236.8, 239+93.9, 31.5+21.7,
                                  188+1, 122+22],
                     'series': list(np.arange(1,11)),
                     'image_path': ['images/bakers/edd.jpg','images/bakers/joanne.jpg','images/bakers/john.jpg',
                                    'images/bakers/frances.jpg','images/bakers/nancy.jpg','images/bakers/nadiya.jpg',
                                    'images/bakers/candice.jpg','images/bakers/sophie.jpg','images/bakers/rahul.jpg',
                                   'images/bakers/david.jpg'],
                     'wiki': ['https://en.wikipedia.org/wiki/Edd_Kimber','https://en.wikipedia.org/wiki/Joanne_Wheatley',
                             'https://en.wikipedia.org/wiki/John_Whaite', 'https://en.wikipedia.org/wiki/Frances_Quinn',
                             'https://en.wikipedia.org/wiki/Nancy_Birtwhistle', 'https://en.wikipedia.org/wiki/Nadiya_Hussain',
                             'https://en.wikipedia.org/wiki/Candice_Brown','https://en.wikipedia.org/wiki/Sophie_Faldo',
                             'https://en.wikipedia.org/wiki/Rahul_Mandal','https://en.wikipedia.org/wiki/David_Atherton_(baker)']
                     })
insta.to_csv('../data/insta.csv',index=False)

**dashboard**

In [28]:
baker_results.iloc[baker_results['star_baker'].idxmax()][['series','baker','star_baker']] # most star bakers

series              5
baker         Richard
star_baker          5
Name: 58, dtype: object

In [54]:
baker_results.iloc[baker_results['technical_winner'].idxmax()][['series','baker','technical_winner']] # best technical baker

series                  3
baker               James
technical_winner        5
Name: 25, dtype: object

In [50]:
(gbbo['Signature Handshake']+gbbo['Showstopper Handshake']).sum() # 34 handshakes given

34

In [63]:
# handshakes_by_season = gbbo.groupby('Season').sum()[['Showstopper Handshake','Signature Handshake']].sum(axis=1)
# handshakes_by_season